In [3]:
import requests
from bs4 import BeautifulSoup

# Request

Ezt az oldalt fogjuk majd scrapelni:  
https://www.gyakorikerdesek.hu/kereses.php?keres=szakkoll%C3%A9gium

In [1]:
URL = "https://www.gyakorikerdesek.hu/kereses.php?keres=szakkoll%C3%A9gium"

GET request

In [4]:
resp = requests.get(URL)

[Status code](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status) mutatja, hogy sikeres volt-e a request.

In [5]:
resp.status_code

200

Ezt pedig errort dob, ha nem jó

In [6]:
resp.raise_for_status()

Na és akkor nézzük meg mit is töltöttünk le valójában

In [18]:
resp.content

b'<!DOCTYPE html>\n<html lang="hu">\n<head>\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<meta charset="utf-8">\n<title>Szakkoll\xc3\xa9gium - Gyakori k\xc3\xa9rd\xc3\xa9sek</title>\n<meta name="description" content="Szakkoll\xc3\xa9gium - K\xc3\xa9rd\xc3\xa9sek a t\xc3\xa9m\xc3\xa1ban. Pl. PhD-sk\xc3\xa9nt k\xc3\xa9s\xc5\x91 szakkol\xc3\xa9giumba jelentkezni?">\n<meta name="keywords" content="szakkoll\xc3\xa9gium, k\xc3\xa9rd\xc3\xa9sek, v\xc3\xa1laszok">\n<meta property="og:title" content="Szakkoll\xc3\xa9gium - Gyakori k\xc3\xa9rd\xc3\xa9sek">\n<meta property="og:description" content="Szakkoll\xc3\xa9gium - K\xc3\xa9rd\xc3\xa9sek a t\xc3\xa9m\xc3\xa1ban. Pl. PhD-sk\xc3\xa9nt k\xc3\xa9s\xc5\x91 szakkol\xc3\xa9giumba jelentkezni?">\n<meta property="og:type" content="website">\n<meta property="og:image" content="//static.gyakorikerdesek.hu/p/logo.jpg">\n<link rel="shortcut icon" href="//static.gyakorikerdesek.hu/favicon.ico">\n<link rel="apple-touch-icon" size

Nem túl informatív. `bytes` típusú objektum.

Jó lenne valamilyen értelmezhetőbb formára hozni. Erre való a `Beautifulsoup` parser.

In [11]:
soup = BeautifulSoup(resp.content, "html5lib")

In [12]:
soup

<!DOCTYPE html>
<html lang="hu"><head>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta charset="utf-8"/>
<title>Szakkollégium - Gyakori kérdések</title>
<meta content="Szakkollégium - Kérdések a témában. Pl. PhD-sként késő szakkolégiumba jelentkezni?" name="description"/>
<meta content="szakkollégium, kérdések, válaszok" name="keywords"/>
<meta content="Szakkollégium - Gyakori kérdések" property="og:title"/>
<meta content="Szakkollégium - Kérdések a témában. Pl. PhD-sként késő szakkolégiumba jelentkezni?" property="og:description"/>
<meta content="website" property="og:type"/>
<meta content="//static.gyakorikerdesek.hu/p/logo.jpg" property="og:image"/>
<link href="//static.gyakorikerdesek.hu/favicon.ico" rel="shortcut icon"/>
<link href="//static.gyakorikerdesek.hu/apple-touch-icon-144x144.png" rel="apple-touch-icon" sizes="144x144"/>
<meta content="#FFFFFF" name="msapplication-TileColor"/>
<meta content="//static.gyakorikerdesek.hu/mstile-144x144.png" name=

# Keresés a DOM-ban

Látjuk hogy vannak `div` típusú elemek `kerdeslista` class-el, amelyek az egyes kérdéseket lehatárolják. 

*Lásd Chrome inspect funkció.*

## Egy elem kijelölése

Jelöljük ki az első ilyen elemet

In [15]:
elem = soup.find("div", attrs={"class", "kerdeslista"})

A `prettify` method segít szépen ábrázolni a DOM-ot.

In [21]:
soup.find('div', attrs={'class': 'kerdeslistak'})

In [17]:
print(elem.prettify())

<div class="flex kerdeslista" id="k11913351" style="flex-wrap: nowrap;">
 <div class="kerdes_normal">
 </div>
 <div class="kerdeslista_szoveg">
  <a href="/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni">
   PhD-sként késő szakkolégiumba jelentkezni?
  </a>
  <br/>
  <span class="kerdes_alatt">
   <a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__magyar-iskolak">
    Közoktatás, tanfolyamok » Magyar iskolák
   </a>
  </span>
 </div>
 <div class="kerdeslista_valasz">
  0
 </div>
</div>



A fenti kijelölést egyébként máshogy is meg lehet csinálni, erre íme pár példa

In [34]:
soup.find("div", class_="kerdeslista")

<div class="flex kerdeslista" id="k11913351" style="flex-wrap: nowrap;"><div class="kerdes_normal"></div><div class="kerdeslista_szoveg"><a href="/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni">PhD-sként késő szakkolégiumba jelentkezni?</a><br/><span class="kerdes_alatt"><a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__magyar-iskolak">Közoktatás, tanfolyamok » Magyar iskolák</a></span></div><div class="kerdeslista_valasz">0</div></div>

In [25]:
soup.select_one("div.kerdeslista")

<div class="flex kerdeslista" id="k11913351" style="flex-wrap: nowrap;"><div class="kerdes_normal"></div><div class="kerdeslista_szoveg"><a href="/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni">PhD-sként késő szakkolégiumba jelentkezni?</a><br/><span class="kerdes_alatt"><a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__magyar-iskolak">Közoktatás, tanfolyamok » Magyar iskolák</a></span></div><div class="kerdeslista_valasz">0</div></div>

Ez utóbbi CSS selector alapján keres. Szerintem ez a legjobb módszer egyébként, univerzálisabban használható.

Segítség hozzá itt:  
https://www.w3schools.com/cssref/css_selectors.php

## Több elem kijelölése

Hasonló az előzőhöz, csak egy listát kapunk vissza a patternnek megfelelő objektumok alapján.

Standard `Beautifulsoup` megoldás

In [26]:
elems = soup.find_all("div", attrs={"class", "kerdeslista"})

In [29]:
elems[0]

<div class="flex kerdeslista" id="k11913351" style="flex-wrap: nowrap;"><div class="kerdes_normal"></div><div class="kerdeslista_szoveg"><a href="/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni">PhD-sként késő szakkolégiumba jelentkezni?</a><br/><span class="kerdes_alatt"><a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__magyar-iskolak">Közoktatás, tanfolyamok » Magyar iskolák</a></span></div><div class="kerdeslista_valasz">0</div></div>

In [42]:
len(elems)

24

CSS selectorral

In [30]:
soup.select("div.kerdeslista")

[<div class="flex kerdeslista" id="k11913351" style="flex-wrap: nowrap;"><div class="kerdes_normal"></div><div class="kerdeslista_szoveg"><a href="/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni">PhD-sként késő szakkolégiumba jelentkezni?</a><br/><span class="kerdes_alatt"><a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__magyar-iskolak">Közoktatás, tanfolyamok » Magyar iskolák</a></span></div><div class="kerdeslista_valasz">0</div></div>,
 <div class="flex kerdeslista sor" id="k11894807" style="flex-wrap: nowrap;"><div class="kerdes_normal"></div><div class="kerdeslista_szoveg"><a href="/kozoktatas-tanfolyamok__magyar-iskolak__11894807-miert-epp-a-harkaly-szerepel-az-elte-bibo-istvan-szakkollegium-logojabanbannere">Miért épp a harkály szerepel az ELTE Bibó István Szakkollégium logójában/bannerében?</a><br/><span class="kerdes_alatt"><a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__magyar-iskolak">Közoktatás, tanfolyamok » Magyar i

# Linkek begyűjtése

Ahhoz hogy be tudjuk gyűjteni a hozzászólásokat, először meg kéne tudni mondani, hogy pontosan hány ilyen kérdés volt és azokat milyen linken lehet elérni.

**Segítség:**  
Ilyenkor elég végig iterálni az oldalakon és begyűjteni minden linket!

Ha megnyitod a második oldalt, akkor látszik, hogy az URL az alábbira változik:  
https://www.gyakorikerdesek.hu/kereses.php?keres=szakkoll%C3%A9gium&oldal=2

Tehát ebből már sejthető, hogy az URL végén az `oldal=2` mondja majd meg, hogy hanyadik oldalon vagyunk jelenleg. Ez egyébként gyakorta így van, mivel az úgynevezett [query stringet](https://en.wikipedia.org/wiki/Query_string) hasznják a legtöbbször.

A query stringek a `request.get` methodnál könnyen megadhatóak egy dictionary-vel:

In [31]:
resp = requests.get(url=URL, params={"oldal": 2})

In [32]:
resp.url

'https://www.gyakorikerdesek.hu/kereses.php?keres=szakkoll%C3%A9gium&oldal=2'

Az egyszerűség kedvéért itt egy függvényt, ami a requestet és a parseolást egyben elvégzi.

In [33]:
def get_soup(url: str, params: dict = None):
    resp = requests.get(url=url, params=params)
    resp.raise_for_status()
    return BeautifulSoup(resp.content, "html5lib")

## Oldalak száma

Már csak azt kéne megtudni, hogy összesen hány oldal lehet.

Ha ránézel az oldalra, látod, hogy gombokkal lehet váltani oldalakat. Ez szuper, mert akkor valószínűleg a HTML-ben is nyoma van ennek. Még látványosabb, hogy az utolsó gomb mindig az utolsó oldalra vezet, tehát nekünk erre lesz majd szükségünk.

Ezt az utolsó elemet egy `div` tárolja, aminek `oldalszamok` a class-e. Ezen belül pedig `a` elemek reprezentálják a gombokat.

In [52]:
soup.select("div.oldalszamok a")

[<a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=2" title="szakkollégium 2.">2</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=3" title="szakkollégium 3.">3</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=4" title="szakkollégium 4.">4</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=5" title="szakkollégium 5.">5</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=6" title="szakkollégium 6.">6</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=7" title="szakkollégium 7.">7</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=8" title="szakkollégium 8.">8</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=9" title="szakkollégium 9.">9</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=10" title="szakkollégium 10.">10</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=2" title="szakkollégium 2.">❯</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=10" title

Látszik, hogy a tetején és az alján is vannak ilyen gombok, de ez nem gond, mert az utolsó elég lesz.

In [35]:
elem = soup.select("div.oldalszamok a")[-1]
elem

<a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=10" title="szakkollégium 10.">❯❯</a>

Ez az az elem, most szedjük ki belőle a releváns információt. A `href` attribútum fogja ez tárolni egy link formájában.

Ezt a `soup.get` methoddal lehet kiszedni.

In [38]:
last_url = elem.get("href")
last_url

'/kereses.php?keres=szakkoll%C3%A9gium&oldal=10'

Végül szedjük ki a string végéről a számot

In [39]:
int(last_url.split("=")[-1])

10

Az egész egy függvényben:

In [40]:
def get_last_page(url: str):
    soup = get_soup(url=url)
    last_elem = elem = soup.select("div.oldalszamok a")[-1]
    last_url = last_elem.get("href")
    return int(last_url.split("=")[-1])

In [41]:
get_last_page(url=URL)

10

## Linkek kigyűjtése

Korábban már láttuk, hogy így néz ki egy kérdés

In [42]:
elem = soup.select("div.kerdeslista")[0]
print(elem.prettify())

<div class="flex kerdeslista" id="k11913351" style="flex-wrap: nowrap;">
 <div class="kerdes_normal">
 </div>
 <div class="kerdeslista_szoveg">
  <a href="/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni">
   PhD-sként késő szakkolégiumba jelentkezni?
  </a>
  <br/>
  <span class="kerdes_alatt">
   <a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__magyar-iskolak">
    Közoktatás, tanfolyamok » Magyar iskolák
   </a>
  </span>
 </div>
 <div class="kerdeslista_valasz">
  0
 </div>
</div>



Nekünk ebből az **id**, a **szöveg** és az **url** fog kelleni.

In [43]:
elem.get("id")

'k11913351'

In [45]:
elem.select_one("div.kerdeslista_szoveg a").get("href")

'/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni'

A szöveget a `soup.get_text` methoddal tudjátok kinyerni. A strip paraméter megadja, hogy levágja-e a felesleges karaktereket a string két oldaláról.

In [49]:
elem.select_one("div.kerdeslista_szoveg a").get_text(strip=True)

'PhD-sként késő szakkolégiumba jelentkezni?'

Egyben az egész egy függvényként

In [50]:
def _parse_listing_elem(elem):
    return {
        "id": elem.get("id"),
        "url": elem.select_one("div.kerdeslista_szoveg a").get("href"),
        "question": elem.select_one("div.kerdeslista_szoveg a").get_text(strip=True),
    }

In [51]:
_parse_listing_elem(elem)

{'id': 'k11913351',
 'url': '/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni',
 'question': 'PhD-sként késő szakkolégiumba jelentkezni?'}

Most csináljunk egy függvényt egy oldal összers kérdésének kinyerésére is.

In [52]:
def parse_listing_page(soup):
    return [_parse_listing_elem(elem) for elem in soup.select("div.kerdeslista")]

In [55]:
parse_listing_page(soup)

[{'id': 'k11913351',
  'url': '/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni',
  'question': 'PhD-sként késő szakkolégiumba jelentkezni?'},
 {'id': 'k11894807',
  'url': '/kozoktatas-tanfolyamok__magyar-iskolak__11894807-miert-epp-a-harkaly-szerepel-az-elte-bibo-istvan-szakkollegium-logojabanbannere',
  'question': 'Miért épp a harkály szerepel az ELTE Bibó István Szakkollégium logójában/bannerében?'},
 {'id': 'k11888739',
  'url': '/kozoktatas-tanfolyamok__magyar-iskolak__11888739-a-benda-kalman-szakkollegium-felveteli-elbeszelgetesen-mi-varhato-milyen-kerdes',
  'question': 'A Benda Kálmán Szakkollégium felvételi elbeszélgetésén mi várható? Milyen kérdések? Milyen témák?'},
 {'id': 'k11854503',
  'url': '/kozoktatas-tanfolyamok__magyar-iskolak__11854503-mcc-mathias-corvinus-collegiumrol-velemeny',
  'question': 'MCC (mathias corvinus collégium)ról vélemény?'},
 {'id': 'k11853338',
  'url': '/kozoktatas-tanfolyamok__specialis-tanfolyamok__

És végül így már végig tudunk iterálni az összes oldalon.

In [56]:
from tqdm import tqdm

In [57]:
pages = list(range(1, get_last_page(url=URL) + 1))
pages

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [58]:
listing_recs = []
for page in tqdm(pages):
    soup = get_soup(url=URL, params={"oldal": page})
    listing_recs.extend(parse_listing_page(soup=soup))

100%|██████████| 10/10 [00:01<00:00,  5.39it/s]


Beolvasás `DataFrame`-be.

In [61]:
import pandas as pd

In [62]:
df_listing = pd.DataFrame(listing_recs)

In [63]:
df_listing

,id,url,question
0,k11913351,/kozoktatas-tanfolyamok__magyar-iskolak__11913...,PhD-sként késő szakkolégiumba jelentkezni?
1,k11894807,/kozoktatas-tanfolyamok__magyar-iskolak__11894...,Miért épp a harkály szerepel az ELTE Bibó Istv...
2,k11888739,/kozoktatas-tanfolyamok__magyar-iskolak__11888...,A Benda Kálmán Szakkollégium felvételi elbeszé...
3,k11854503,/kozoktatas-tanfolyamok__magyar-iskolak__11854...,MCC (mathias corvinus collégium)ról vélemény?
4,k11853338,/kozoktatas-tanfolyamok__specialis-tanfolyamok...,Lenne esélyem szakkollégiumba?
...,...,...,...
122,k1123436,/kozoktatas-tanfolyamok__tanulasi-lehetosegek_...,Szakkollégium = NO szabadidő?
123,k1113072,/kozoktatas-tanfolyamok__egyeb-kerdesek__11130...,Van valami tapasztalatotok a Magyar Protestáns...
124,k1112838,/kultura-es-kozosseg__vallas__1112838-van-vala...,Van valami tapasztalatotok a Magyar Protestáns...
125,k1039145,/kozoktatas-tanfolyamok__magyar-iskolak__10391...,PPKE-JÁK-ra jelentkeztem és fel is vettek. Csa...


# Szövegek kiszedése

In [64]:
listing_recs[0]["url"]

'/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni'

Ilyen URL-eket kaptunk a korábbi lépésben. Ezeket úgy kell értelmezni, hogy az alap URL (https://www.gyakorikerdesek.hu) utáni részt jelölik:
https://www.gyakorikerdesek.hu/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni

Nézzünk meg jobban egy ilyen oldalt. A szövegeket a `kerdes_kerdes` vagy `valasz_valasz` class-el rendelkező `div` elemek tartalmazzák.

In [65]:
BASE_URL = "https://www.gyakorikerdesek.hu"
BASE_URL + listing_recs[0]["url"]

'https://www.gyakorikerdesek.hu/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni'

In [66]:
soup = get_soup(url=BASE_URL + listing_recs[4]["url"])

Válasszuk ki a megfelelő elemeket. A CSS selectorban a vessző az uniót jelöli.

In [68]:
elems = soup.select("div.kerdes_kerdes, div.valasz_valasz")

Nézzük meg az első két ilyen elemet.

In [355]:
print(elems[0].prettify())

<div class="kerdes_kerdes">
 <div style="width: 100%; margin-bottom: 10px; text-align: right;">
  <input class="check" id="chkb" onclick='location.href="/belepes"; return false;' type="checkbox"/>
  Figyelt kérdés
 </div>
 <p style="margin: 0 0 7px 0;">
  Sziasztok!
 </p>
 <p style="margin: 7px 0 7px 0;">
  Most fogom az egyetemet kezdeni. Nem voltam kitűnő és versenyekre sem jártam, az érettségim viszont mindenből 90% felett lett.
 </p>
 <br/>
 <p style="margin: 7px 0 7px 0;">
  Emellett 1 felső és 2 középfokú nyelvvizsgám van +2 idegen nyelvet b2 szinten beszélek.
 </p>
 <br/>
 <p style="margin: 7px 0 7px 0;">
  Inkább a beszéd, mintsem az írás az erősségem.
 </p>
 <p style="margin: 7px 0 7px 0;">
  Az ELTE szakkollégiumai közül melyiket javasolnátok nekem, ahová lenne is esélyem bekerülni?
 </p>
 <p style="margin: 7px 0 7px 0;">
  Előre is köszönöm szépen a segítséget.
 </p>
 <br/>
</div>



In [297]:
print(elems[1].prettify())

<div class="kerdes_kerdes" style="padding-bottom: 10px;">
 <div class="flex" style="flex-wrap: nowrap;">
  <div id="valasz73717835" style="min-width: 740px;">
   <p style="margin: 0 0 7px 0;">
    Erre gondolok:
   </p>
   <p style="margin: 7px 0 7px 0;">
    <a onclick='alert("A link bejelentkezés után lesz kattintható.");' title="Ha bejelentkezel, a link kattinthatóvá válik">
     [link]
    </a>
   </p>
  </div>
 </div>
</div>



In [75]:
def parse_msg_text(elem):
    if elem.select_one("div input.check"):
        if elem.select_one("p"):
            return "\n".join([p.get_text(strip=True) for p in elem.select("p, br")])
        else:
            [e.decompose() for e in elem.select("div")]
            return elem.get_text(strip=True)
    else:
        return elem.select("div.flex div")[-1].get_text(strip=True)

Végül írjunk egy függvényt, ami feldolgoz egy kommentet.

In [76]:
def parse_msg(elem):
    return {"comment": parse_msg_text(elem)}

És egy olyat is, ami az adott oldal összes kommentjét feldolgozza.

In [77]:
def parse_msg_page(soup):
    elems = soup.select("div.kerdes_kerdes, div.valasz_valasz")
    return [parse_msg(e) for e in elems]

In [78]:
parse_msg_page(soup)

[{'comment': 'Sziasztok!\nMost fogom az egyetemet kezdeni. Nem voltam kitűnő és versenyekre sem jártam, az érettségim viszont mindenből 90% felett lett.\n\nEmellett 1 felső és 2 középfokú nyelvvizsgám van +2 idegen nyelvet b2 szinten beszélek.\n\nInkább a beszéd, mintsem az írás az erősségem.\nAz ELTE szakkollégiumai közül melyiket javasolnátok nekem, ahová lenne is esélyem bekerülni?\nElőre is köszönöm szépen a segítséget.\n'},
 {'comment': 'Melyik szakkolikat nézted vagy egyáltalán milyen szakos vagy? Több szakkoli is van, nem mindegyikbe jelentkezhetnek minden szakról.'},
 {'comment': 'ELTE Eötvös József Kollégiumot néztem.'},
 {'comment': 'És milyen szakos vagy?'},
 {'comment': 'Nemzetközi tanulmányok'},
 {'comment': 'Navratil Ákos Szakkollégium akkor az igazán neked való, bár lehet az EC-ben is van olyan műhely, ahova tudsz jelentkezni, bár az EC-be nehezebb bekerülni.'}]

Végül iteráljunk végig az összes thread-en és szedjük ki a kommenteket.

In [79]:
listing_recs[0]

{'id': 'k11913351',
 'url': '/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni',
 'question': 'PhD-sként késő szakkolégiumba jelentkezni?'}

Az id-k miatt `DataFrame`-et használunk. Erről később lesz szó.

In [80]:
comments = []
for listing in tqdm(listing_recs):
    soup = get_soup(url=BASE_URL + listing["url"])
    comment_recs = parse_msg_page(soup)
    comments.append(pd.DataFrame(comment_recs).assign(thread_id=listing["id"]))

100%|██████████| 127/127 [00:30<00:00,  4.22it/s]


In [81]:
df_comments = pd.concat(comments)
df_comments

,comment,thread_id
0,"Amikor alapszakos és mesterszakos voltam, túls...",k11913351
1,"Ja, természetesen most már nem lehet jelentkez...",k11913351
0,A szakkollégium 2008-ban alapított Harkály-díj...,k11894807
1,Erre gondolok:[link],k11894807
2,"Megkaptam a választ máshonnan, ha valakit még ...",k11894807
...,...,...
3,annak a szaknak a kimondott kolesza ahova felv...,k1039145
4,"Nem, nem azt jelenti. A szakkollégium név arra...",k1039145
5,Na de akkor csak abba a koleszba felvételizhet...,k1039145
0,"Lakott már vki ott, és milyen hely? Közösség jó?",k327099


In [370]:
df_listing

,id,url,question
0,k11913351,/kozoktatas-tanfolyamok__magyar-iskolak__11913...,PhD-sként késő szakkolégiumba jelentkezni?
1,k11894807,/kozoktatas-tanfolyamok__magyar-iskolak__11894...,Miért épp a harkály szerepel az ELTE Bibó Istv...
2,k11888739,/kozoktatas-tanfolyamok__magyar-iskolak__11888...,A Benda Kálmán Szakkollégium felvételi elbeszé...
3,k11854503,/kozoktatas-tanfolyamok__magyar-iskolak__11854...,MCC (mathias corvinus collégium)ról vélemény?
4,k11853338,/kozoktatas-tanfolyamok__specialis-tanfolyamok...,Lenne esélyem szakkollégiumba?
5,k11851503,/kozoktatas-tanfolyamok__magyar-iskolak__11851...,Az Eötvös József Szakkollégiumra kinek van esé...
6,k11837483,/kozoktatas-tanfolyamok__magyar-iskolak__11837...,ELTE szakkollégiumokról mit lehet tudni? Hogya...
7,k11832490,/kozoktatas-tanfolyamok__egyeb-kerdesek__11832...,"Melyiket ajánlanátok inkább, az ELTE Márton Ár..."
8,k11800923,/kozoktatas-tanfolyamok__magyar-iskolak__11800...,Az ELTE Illyés Sándor Szakkollégiumába mennyir...
9,k11792228,/kozoktatas-tanfolyamok__tanulasi-lehetosegek_...,ELTE IK programtervező informatikus MSc vélemé...
